In [21]:
import numpy as np
import pandas as pd
import string,re,nltk
from nltk import WordNetLemmatizer
stopwords=nltk.corpus.stopwords.words('english')
wn=WordNetLemmatizer()

In [5]:
with open("C:/Users/Akshat/Desktop/t.txt") as file:
    lines = []
    for line in file:
        # The rstrip method gets rid of the "\n" at the end of each line
        lines.append(line.rstrip().split(","))

In [11]:
with open('C:/Users/Akshat/Desktop/t.txt', 'r') as myfile:
    data=myfile.read().replace('\n', '')

In [12]:
type(data)

str

In [14]:
def standardize_text(text_field):
    text_field = text_field.replace(r"http\S+", "")
    text_field = text_field.replace(r"http", "")
    text_field = text_field.replace(r"@\S+", "")
    text_field = text_field.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    text_field = text_field.replace(r"@", "at")
    text_field = text_field.lower()
    text_field = text_field.replace('\s+', ' ')
    text_field = text_field.replace("\'", "")
    return text_field

In [15]:
data=standardize_text(data)

In [16]:
def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [wn.lemmatize(word) for word in tokens if word not in stopwords]
    return text

In [17]:
import string,re

In [22]:
a=clean_text(data)

In [52]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation,NMF

In [47]:

# Initialize variables
n_samples = 2000
n_features = 1000
n_topics = 5
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                stop_words='english')
tf = tf_vectorizer.fit_transform(a)

lda = LatentDirichletAllocation(n_components=n_topics, max_iter=5,
                                learning_method='online', learning_offset=50.,
                                random_state=0)
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=5, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [44]:
def display_topics(model, feature_names, no_top_words=10):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [45]:
display_topics(lda, tf_vectorizer.get_feature_names(), 10)

Topic 0:
data analytics created 2016 2014 trend software automated reporting dashboard
Topic 1:
analysis reporting public proof decrease jul hour science spark created
Topic 2:
model resulting business rate jun cleaning insurance dec response tool
Topic 3:
project microstrategy warehouse process intelligence gathering oracle present science rate
Topic 4:
mstr complaint algorithm new concept modeling research sql intelligence jul
Topic 5:
using time 2015 july senior team business jul response trend
Topic 6:
resolution client spark presales research 2015 new hour time resulting
Topic 7:
sql team organization apache building dashboard 2018 automated hr business
Topic 8:
creating application custom sap 2018 present warehouse software trend cleaning
Topic 9:
python server learning assistant hr resolution tool using data spark


In [49]:
display_topics(lda, tf_vectorizer.get_feature_names(), 10)

Topic 0:
data using team reporting client 2016 july 2014 response trend
Topic 1:
analysis time complaint created spark apache public proof decrease senior
Topic 2:
model 2015 resulting rate business jun new application insurance dec
Topic 3:
project organization microstrategy creating resolution warehouse process intelligence cleaning presales
Topic 4:
python sql mstr server analytics algorithm concept jul hour modeling


In [56]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(a)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [36]:
import gensim
from gensim import corpora

In [61]:
bigram = gensim.models.Phrases(a, min_count=5, threshold=10) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[a], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

C:\Users\Akshat\Anaconda3\lib\site-packages\gensim\models\phrases.py:431: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [75]:
def make_bigrams(texts):
    return [bigram_mod[texts]]

In [68]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [69]:
print(data_words[:1])

[[]]


In [76]:
data_words= make_bigrams(a)

In [92]:
# data_words

In [78]:
id2word = corpora.Dictionary(data_words)

In [79]:
corpus = [id2word.doc2bow(text) for text in data_words]

In [91]:
# [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [99]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [100]:
x=lda_model.show_topics(num_topics=10, num_words=5,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

# #Below Code Prints Topics and Words
# for topic,words in topics_words:
#     print(str(topic)+ "::"+ str(words))
# print()

#Below Code Prints Only Words 
for topic,words in topics_words:
    print(" ".join(words))

data using sql python reporting
data using python sql team
data using python sql reporting
data using team sql python
data using python analysis sql
data using python sql analysis
data using python sql resulting
data using python sql business
data using python sql team
data using python sql team


In [ ]:
display_topics

In [84]:
import pprint
pprint.pprint(lda_model.print_topics())
#doc_lda = lda_model[corpus]

[(0,
  '0.004*"data" + 0.004*"using" + 0.004*"python" + 0.004*"sql" + '
  '0.003*"reporting" + 0.003*"analysis" + 0.003*"team" + 0.003*"2015" + '
  '0.003*"server" + 0.003*"model"'),
 (1,
  '0.004*"data" + 0.004*"using" + 0.004*"python" + 0.003*"sql" + 0.003*"team" '
  '+ 0.003*"reporting" + 0.003*"mstr" + 0.003*"analysis" + 0.003*"business" + '
  '0.003*"server"'),
 (2,
  '0.004*"data" + 0.004*"using" + 0.003*"python" + 0.003*"sql" + '
  '0.003*"reporting" + 0.003*"analysis" + 0.003*"team" + 0.003*"model" + '
  '0.003*"resulting" + 0.003*"time"'),
 (3,
  '0.003*"data" + 0.003*"using" + 0.003*"python" + 0.003*"team" + 0.003*"sql" '
  '+ 0.003*"analysis" + 0.003*"server" + 0.003*"business" + 0.003*"time" + '
  '0.003*"reporting"'),
 (4,
  '0.004*"data" + 0.004*"using" + 0.003*"python" + 0.003*"analysis" + '
  '0.003*"sql" + 0.003*"team" + 0.003*"2015" + 0.003*"reporting" + '
  '0.003*"project" + 0.003*"business"'),
 (5,
  '0.042*"data" + 0.030*"using" + 0.021*"python" + 0.019*"sql" + '


In [89]:
from gensim import corpora, models, utils

In [93]:
# mallet_path = 'C:/Users/Akshat/Downloads/mallet-2.0.8/mallet-2.0.8/bin/mallet' # update this path
# ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)

In [97]:
from gensim.models import CoherenceModel
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.01821547438

Coherence Score:  0.244629476127
